# MNIST DATASET 

MNIST refers to handwritten digit recognition. The dataset provides 70,000 images (28x28 pixels) of handwritten digits (1 digit per image). The goal is to write an algorithm that detects which digit is written. Since there are only 10 digits (0, 1, 2, 3, 4, 5, 6, 7, 8, 9), this is a classification problem with 10 classes. 

In [29]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds

In [30]:
mnist_dataset,mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

In [31]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

In [32]:
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

In [33]:
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [34]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [35]:
BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

In [36]:
BATCH_SIZE=100
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

In [37]:
input_size = 784
output_size = 10
hidden_layer_size = 50

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')   
                            ])

### optimizer and loss function

In [38]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training

In [39]:
NUM_EPOCHS = 5

model.fit(train_data, epochs = NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 10s - loss: 0.4173 - accuracy: 0.8820 - val_loss: 0.2189 - val_accuracy: 0.9365
Epoch 2/5
540/540 - 8s - loss: 0.1891 - accuracy: 0.9450 - val_loss: 0.1614 - val_accuracy: 0.9517
Epoch 3/5
540/540 - 6s - loss: 0.1437 - accuracy: 0.9579 - val_loss: 0.1353 - val_accuracy: 0.9607
Epoch 4/5
540/540 - 6s - loss: 0.1198 - accuracy: 0.9645 - val_loss: 0.1159 - val_accuracy: 0.9667
Epoch 5/5
540/540 - 6s - loss: 0.1017 - accuracy: 0.9697 - val_loss: 0.1043 - val_accuracy: 0.9705


## Test the model

In [40]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 1s 1s/step - loss: 0.1190 - accuracy: 0.9645


In [41]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.12. Test accuracy: 96.45%
